In [ ]:
## made .txt file of same name, gave me txt version of page with metadata, interesting!
# or not? ican open txt file of it either way

In [239]:
# imports
import pandas as pd
import numpy as np
import spotipy #pip install spotipy first
import os
from client import *
from spotipy.oauth2 import SpotifyClientCredentials # required to access user data
import spotipy.util as util

### Spotify Authorization

In [240]:
user= userID
secret = secret
cid = clientID

In [241]:
def get_token(scope=None): #get token and set scope later if want
    redirect_uri = "http://localhost:4000"
    token = util.prompt_for_user_token(user, scope, cid, secret, redirect_uri)
    return token

In [242]:
token = get_token(scope='playlist-modify-private') # can make public later

sp = spotipy.Spotify(auth=token)

### Initialize the Playlist and Main Dataframe

In [243]:
# run once, use playlist_id to ADD the songs to
def initialize_playlist():
    playlist = sp.user_playlist_create(user, 'AutoRepeatAll', public = False)
    playlist_id = playlist["id"]
    return playlist_id

In [244]:
AutoRepeatAll_id = initialize_playlist()

In [245]:
# playlist urls to get data from
playlist_urls =['37i9dQZF1EpoJIsGQLtjPv', '37i9dQZF1EppjlneYJWXxZ', '37i9dQZF1Epf11aYgU5Q4u', '37i9dQZF1EpoxPdVWcv3CP', '37i9dQZF1EpwZ0YRZwMO7Y', '37i9dQZF1EpoCGPknzdJwy', '37i9dQZF1EphQVRnk9uGI6', '37i9dQZF1Epg86odGM5hPi', '37i9dQZF1EpL8wYDpLK6E6', '37i9dQZF1EpfkWyPBlEQKr']
creator = "spotify"

In [246]:
# make df to hold items to be added to playlist?
columns = ["artist", "album", "track_name", "track_id"]
main_df = pd.DataFrame(columns = columns)

### Get the songs from the playlists

In [248]:
# gets playlist tracks from single playlist and makes a df with track data for single playlist

def playlist_to_df(creator, playlist_id):
    playlist_df = pd.DataFrame(columns=columns)
    playlist_items = sp.user_playlist_tracks(creator, playlist_id)["items"] # returns item detail for each song
    for track in playlist_items:
        track_features = {} # make dict for each track with appropriate info
        track_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        track_features["album"] = track["track"]["album"]["name"]
        track_features["track_name"] = track["track"]["name"]
        track_features["track_id"] = track["track"]["id"]
        
        track_df = pd.DataFrame(track_features, index=[0]) # make df from dict for each item
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True) # add it to the ma
        
    return playlist_df
#https://www.linkedin.com/pulse/extracting-your-fav-playlist-info-spotifys-api-samantha-jones/

In [ ]:
# make playlist df for all playlists in url_list
# main_df = pd.concat('playlist_df_list')

In [249]:
def make_playlist_df_list(playlist_urls):
    playlist_df_list = []
    for playlist in playlist_urls:
        playlist_df_list.append(playlist_to_df(creator, playlist))
    return playlist_df_list

In [251]:
df_playlist_list = make_playlist_df_list(playlist_urls) # list of dfs

In [253]:
# concats all track info into one main_df
def add_playlist_dfs_to_main_df():
    main_df = pd.concat(df_playlist_list, axis=0, ignore_index=True)
    return main_df

In [255]:
add_playlist_dfs_to_main_df()

,artist,album,track_name,track_id
0,The Smiths,Rank,"Still Ill - Live in London, 1986",1EU12yfayMBmqQy3Q92kn1
1,Kanye West,The Life Of Pablo,FML,30Zcjs7pgEkmJA1lEbGSWT
2,Yowler,Black Dog in My Path,Where Is My Light?,2IRbktKXjRKTTHn6H83nX3
3,Babyshambles,Down In Albion,Fuck Forever,41Z2joQXOwZRBB8V6zk0A8
4,Starflyer 59,Gold,Stop Wasting Your Whole Life/Messed Up And Dow...,3e7kxFnCGHrotr8h8fQleP
...,...,...,...,...
295,Thom Yorke,The Eraser,Skip Divided,3oC3iuyvwfaTCOBWL6YOYj
296,Pitcher56,Her Abiding Memory,Another Season,7lQec0pXsniGVJaOnidjUi
297,Radiohead,The Bends,The Bends,7oDFvnqXkXuiZa1sACXobj
298,Cocteau Twins,Treasure,Lorelei,1ZnXuijpwEBjhNxpEnb8WH


### get song info from main_df

In [219]:
# gets playlist tracks from single playlist and makes a df with track data for single playlist

def call_playlist(creator, playlist_id):
#     playlist_details = ["artist", "album", "track_name", "track_id"]
#     playlist_df = pd.DataFrame(columns = playlist_details)
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        playlist_features = {}
        
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        track_df = pd.DataFrame(playlist_features, index=[0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df
#https://www.linkedin.com/pulse/extracting-your-fav-playlist-info-spotifys-api-samantha-jones/

In [185]:
def make_main_playlist(play_list):
    playlist = sp.user_playlist_create(user, 'autoRepeatAll', public = False)
    playlist_id = playlist["id"]
    
#     playlist_details = ["artist", "album", "track_name", "track_id"]
#     df_i_think = pd.DataFrame(columns = playlist_details)
    
#     playlist = sp.user_playlist_tracks(creator, playlist_id)
    
    playlist_details = ["artist", "album", "track_name", "track_id"]
    track_list = pd.DataFrame(columns=playlist_details)
    for url in playlist_urls[0:3]:
        track_list = track_list.append(call_playlist('spotify', url))
    all_tracks = track_list["track_id"].tolist()
    sp.user_playlist_add_tracks(user, playlist_id = playlist_id, tracks = all_tracks)

    return track_list
    

In [186]:
make_main_playlist(play_list)

,artist,album,track_name,track_id
0,Kanye West,The Life Of Pablo,FML,30Zcjs7pgEkmJA1lEbGSWT
1,The Strokes,Room On Fire,Reptilia,57Xjny5yNzAcsxnusKmAfA
2,Ian Pooley,Since Then. 2017,900 Degrees,3nBF7oaQBAY6M2eVuUCnYn
3,Yowler,The Offer,Go,4BcYAmmTTqvGfHqBrg3v4S
4,Julie Doiron,Julie Doiron & the Wooden Stars,Sweeter,16L2DWEkwTF9OPfb9TiaUt
...,...,...,...,...
25,Clyde Cyrus,Her,Her,3s7axnNhQTrrpzDZwH0V61
26,The Rolling Stones,It’s Only Rock ’N’ Roll (But I Like It) / Rip ...,It’s Only Rock ’N’ Roll (But I Like It) - Live...,1U4pHHISIyKRKFVLBBebau
27,Soccer Mommy,Shotgun,Shotgun,35PUhhwZL4HhmuqgOeFcfg
28,Riz,Dance with Me,Dance with Me,1yZ8dKi13txFkMDSySkQIf


In [ ]:
def make_main_playlist()

In [145]:
df_list_i_think

[                             artist  \
 0                        Kanye West   
 1                       The Strokes   
 2                        Ian Pooley   
 3                            Yowler   
 4                      Julie Doiron   
 5                         Cindy Lee   
 6                        Ian Pooley   
 7   Herb Alpert & The Tijuana Brass   
 8                        The Smiths   
 9                      Julie Doiron   
 10                     Babyshambles   
 11                           Yowler   
 12                     Starflyer 59   
 13                  Various Artists   
 14                           Mitski   
 15                          EASYFUN   
 16                     Starflyer 59   
 17                    Axel Le Baron   
 18                 Jackson C. Frank   
 19                          Chevals   
 20                    Dave Van Ronk   
 21                       Weatherday   
 22                  Various Artists   
 23                    Axel Le Baron   


In [111]:
df1 = call_playlist('spotify', '37i9dQZF1EpfkWyPBlEQKr')

In [115]:
track_list = df1["track_id"].tolist()

In [128]:
def make_playlist(df, track_list):
    playlist = sp.user_playlist_create(user, 'autoRepeat', public = False)
    playlist_id = playlist["id"]
    
    sp.user_playlist_add_tracks(user, playlist_id=playlist_id, tracks=track_list)
    return playlist_id

In [129]:
make_playlist(df1, track_list)

'7q1aLfR34kcOI3jhwMM1ah'

In [ ]:
def make_playlist(url_list):
    make_playlist = sp.user_playlist_create(user, 'autoRepeat', public=False)
    mp_id = make_playlist["id"]
    
    for url in url_list.iterrows():
        title_x = j.title
        artist_x = j.artist
        search_query = title_x + ' ' + artist_x
        search_results = sp.search(q=search_query, limit=1)
        try:
            uri = [search_results["tracks"]["items"][0]["uri"]]
            sp.playlist_add_items(playlist_id=mp_id, items=uri)
        except: pass
    return mp_id

In [144]:
## for url in list, call playlist
# def make_main_playlist(creator, play_list):
#     playlist_details = ["artist", "album", "track_name", "track_id"]
#     df_i_think = pd.DataFrame(columns = playlist_details)
    
#     playlist = sp.user_playlist_tracks(creator, playlist_id)
    
#     for url in play_list:
#         df_i_think = pd.concat([playlist_df, call_playlist('spotify', playlist_df[id])])
    

In [138]:
## read in pickle, get list of playlists from pickle
# pickle doesnt have urls in proper format so won't work yet

playlist_list = pd.read_pickle(r'./data/onRepeatList.pkl')

play_list = playlist_list['url'].tolist()